In [1]:
import jutil
import numpy as np

In [2]:
# 输入学生学号,要预测的课程X,返回该门课是否挂科

In [3]:
# 1.找出X的前置课程,确定学生的专业和年级
sql = "select grade,speciality_code,course_code,course_name,term_order,mark,pmark from view_stu_course_mark where student_id='2014112135'"
stu = jutil.load_pd_df(sql)
stu.head()

,course_code,course_name,grade,mark,pmark,speciality_code,term_order
0,0471004,面向对象程序设计,2014,10,90.0,0408,2
1,0471005,面向对象程序设计实验,2014,10,96.0,0408,2
2,0471024,计算机网络工程实验,2014,6,76.0,0408,4
3,0471046,J2EE体系结构及程序设计,2014,10,95.0,0408,4
4,0471054,程序语言综合课程设计,2014,10,90.0,0408,2


In [48]:
course_name = '计算机网络'
a = stu[stu['course_name']==course_name]
a

,course_code,course_name,grade,mark,pmark,speciality_code,term_order
33,3223700,计算机网络,2014,9,88.0,0408,4


In [37]:
grade = a['grade'].values[0]
term_order = a['term_order'].values[0]
speciality_code = a['speciality_code'].values[0]
course_code = a['course_code'].values[0]

In [38]:
pre_course = stu[stu['term_order']<term_order]

In [49]:
course_codes = list(pre_course['course_code'].values)
course_codes.append(course_code)
# course_codes
course_names = list(pre_course['course_code'].values)
course_names.append(course_name)

In [20]:
# 找前一个年级的成绩数据来预测

In [33]:
pre_grade = str(int(grade) - 1)
sql = "select student_id,course_name,course_code,pmark,mark from view_stu_course_mark where speciality_code='%s' and grade='%s'" \
    %(speciality_code,pre_grade)
df = jutil.load_pd_df(sql)
df = df.drop_duplicates(subset=['student_id','course_code'])
df.head()

,course_code,course_name,mark,pmark,student_id
0,0471004,面向对象程序设计,6,75.0,20112807
1,0471005,面向对象程序设计实验,3,67.0,20112807
2,3045600,数字电子技术B,10,96.0,20112807
3,3232400,计算机图形学,5,74.0,20112807
4,3244237,嵌入式系统开发,5,72.0,20112807


In [25]:
# 选出需要的课程


In [47]:
stu_ids = df[df['course_code']==course_code]['student_id'].values
d1 = df[df['student_id'].isin(stu_ids)]
dd = d1[d1['course_code'].isin(course_codes)]

In [58]:
now_names = set(dd['course_name'].values)
len(now_names)

61

In [59]:

gd = dd.groupby(['student_id'],as_index=False)
d_arr = dict() # key存的是课程名,value是存放成绩的数组,这样竖着看就是一个学生的成绩列表了,也就是转置操作
d_arr2 = dict() 
for name ,g in gd:
    d_m = dict()
    d_m2 = dict()
    for i,row in g.iterrows():
        d_m[row['course_name']] = row['pmark']
        d_m2[row['course_name']] = row['mark']
    for c in now_names:
        d_arr[c] = d_arr.get(c,[])
        d_arr[c].append(d_m.get(c,60))
        d_arr2[c] = d_arr2.get(c,[])
        d_arr2[c].append(d_m2.get(c,1))
courses = d_arr.keys()
mt_old = [arr for arr in d_arr.values()]
mt2_old = [arr for arr in d_arr2.values()]
mt = np.transpose(mt_old)
mt

array([[60., 81., 61., ..., 75., 71., 85.],
       [68., 60., 60., ..., 83., 22., 85.],
       [76., 35., 75., ..., 72., 72., 85.],
       ...,
       [78., 60., 83., ..., 82., 60., 85.],
       [81., 60., 87., ..., 86., 60., 85.],
       [73., 60., 68., ..., 92., 60., 60.]])

In [94]:
# 要将mt矩阵进行拆分成数据和标签(预测的那门课)
y = d_arr[course_name]
X = np.transpose([v for k,v in d_arr.items() if not k==course_name])
course_old = [k for k in d_arr.keys() if not k==course_name]

In [68]:
X.shape

(116, 60)

In [65]:
len(y)

116

In [134]:
label = []
for m in y:
    if m>65:
        label.append(1)
    else:
        label.append(0)

In [71]:
# 对X做特征选取和降维操作
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [135]:
sel = SelectKBest(chi2, k=20)
X_new = sel.fit_transform(X, label)
indice = sel.get_support()

In [136]:
X_new

array([[61., 64., 76., ..., 85., 81., 75.],
       [60., 60., 46., ..., 65., 83., 83.],
       [75., 74., 76., ..., 68., 81., 72.],
       ...,
       [83., 60., 74., ..., 64., 71., 82.],
       [87., 60., 71., ..., 85., 89., 86.],
       [68., 60., 62., ..., 63., 79., 92.]])

In [74]:
# 进行交叉验证下

In [75]:
from sklearn.model_selection import train_test_split

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.33)

In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score

In [144]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [145]:
y_pred = clf.predict(X_test)

In [146]:
accuracy_score(y_test,y_pred)

0.6923076923076923

In [109]:
import graphviz

In [148]:
dot_data = tree.export_graphviz(clf, out_file=None,class_names=['挂科','正常'],feature_names=course_old) 
graph = graphviz.Source(dot_data)
graph.render("jimo") 

'jimo.pdf'

In [125]:
course_choose = [course_old[i] for i in range(len(course_old)) if indice[i]]

In [96]:
# 进行预测
course_choose

['马克思主义基本原理',
 '离散数学A',
 '线性代数B',
 '计算方法C',
 '高级语言程序设计',
 '软件项目管理',
 '算法设计与分析',
 '计算机组成实验C',
 '信息检索',
 '数据结构实验',
 '面向对象程序设计',
 '高等数学Ⅰ',
 '英语Ⅱ',
 '面向对象程序设计实验',
 '高等数学Ⅱ',
 '毛泽东思想和中国特色社会主义理论体系概论Ⅰ',
 '概率论与数理统计B',
 '大学物理BⅡ',
 '微机与接口技术实验',
 '操作系统']

In [150]:
xx = []
for c in course_old:
    v = stu[stu['course_name']==c]['pmark'].values
    if len(v)>0:
        xx.append(v[0])
    else: xx.append(60)
x_n = [xx]

In [151]:
clf.predict(x_n)

array([1])

In [131]:
x_1 = [[90 for i in range(20)]]
clf.predict(x_1)

array([0])